In [ ]:
execfile ("D:/measuring/analysis/scripts/setup_analysis.py")

import os, sys 
from os import listdir 
from os.path import isfile, join 
import numpy as np 
import matplotlib.pyplot as plt 
import h5py
from analysis.lib.tools import toolbox

from analysis.lib.fitting import fit, common

%matplotlib inline 

from analysis.lib.pq import pq_tools
reload(pq_tools)

In [ ]:
tb.get_msmt_fp(tb.latest_data())

In [ ]:
# fps=tb.get_all_msmt_filepaths(folder = folder)
# pqf = pq_tools.pqf_from_fp(fps[0], rights = 'r+')
# print fps


lt3_fp = r'd:\\measuring\\data\\20170420\\153117_PQ_single_click_ent_111no2_SIL2_test_pulses\\153117_PQ_single_click_ent_111no2_SIL2_test_pulses.hdf5'
lt4_fp = r'd:\\measuring\\data\\20170420\\152859_PQ_single_click_ent_111no2_SIL2_test_pulses\\152859_PQ_single_click_ent_111no2_SIL2_test_pulses.hdf5'

lt3_pqf = pq_tools.pqf_from_fp(lt3_fp, rights = 'r+')
lt4_pqf = pq_tools.pqf_from_fp(lt4_fp, rights = 'r+')

print 'LT3'
print lt3_fp

print 'LT4'
print lt4_fp


## Check delay time

In [ ]:
def xcorr(a,v):
            
    xaxis = np.arange(len(a)) - len(a)/2
    return xaxis,np.correlate(np.array(a)-np.mean(a),np.array(v)-np.mean(v),'same')

start = 1724e3
length = 70e3

(lt3_h0, b0), (lt3_h1, b1) = pq_tools.get_photon_hist(lt3_pqf,start = start, length = length, hist_binsize = 20)
(lt4_h0, b0), (lt4_h1, b1) = pq_tools.get_photon_hist(lt4_pqf,start = start, length = length, hist_binsize = 20)

print "APD Channel 0"
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(b0[:-1],lt3_h0)
plt.plot(b0[:-1],lt4_h0)
plt.show()
plt.close('all')

print "APD Channel 1"
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(b1[:-1],lt3_h1)
plt.plot(b1[:-1],lt4_h1)
plt.show()
plt.close('all')

xaxis,xcorr00=xcorr(lt3_h0,lt4_h0)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr00)
plt.show()
plt.close('all')
i = np.argmax(xcorr00)
print 'Channel 0 delay is ', xaxis[i], i


xaxis,xcorr11=xcorr(lt3_h1,lt4_h1)
xaxis = (b1[1]-b1[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr11)
plt.show()
plt.close('all')
i = np.argmax(xcorr11)
print 'Channel 0 delay is ', xaxis[i], i

# Apply some interpolation and smoothing to better estimate the top of the cross correlation.

In [ ]:
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

# interpolate + smooth
xnew = np.arange(-2000, 2000, 1)
itp = interp1d(xaxis,xcorr00, kind='linear')
window_size, poly_order = 701, 3
ynew = savgol_filter(itp(xnew), window_size, poly_order)

plt.plot(xnew,ynew)
plt.show()
plt.close('all')
i = np.argmax(ynew)
print 'Channel 0 delay is ', xnew[i]

itp = interp1d(xaxis,xcorr11, kind='linear')
window_size, poly_order = 701, 3
ynew = savgol_filter(itp(xnew), window_size, poly_order)

plt.plot(xnew,ynew)
plt.show()
plt.close('all')
i = np.argmax(ynew)
print 'Channel 1 delay is ', xnew[i]